In [46]:
import requests
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import svm

In [47]:
def moving_average(x, w):
        moving_avg = np.convolve(x, np.ones(w), 'valid') / w
        padding = np.full_like(np.empty(w), np.nan)
        return np.insert(moving_avg, 0, padding)

def moving_avg_diff(short, long):
        return (short - long) / long


In [48]:
def get_X_y(df):
    
    min_period = 24*31 + 1
    
    percent_change = (df["Close"] - df["Close"].shift(1)) / df["Close"].shift(1)
    print(percent_change[:5])

    y = percent_change.copy()
    
    print(len(y))
    
    ## TEMP
    y = np.zeros(percent_change.shape)
    y[np.where(percent_change > 0)] = 1

    moving_average_1_hours = moving_average(df["Close"], 1)
    moving_average_2_hours = moving_average(df["Close"], 2)
    moving_average_3_hours = moving_average(df["Close"], 3)
    moving_average_12_hours = moving_average(df["Close"], 12)
    moving_average_24_hours = moving_average(df["Close"], 24)
    moving_average_48_hours = moving_average(df["Close"], 48)
    moving_average_1_week = moving_average(df["Close"], 24*7)
    moving_average_1_month = moving_average(df["Close"], 24*31)
    
    print(len(moving_average_1_month))
    
    mv_1h_2h = moving_avg_diff(moving_average_1_hours, moving_average_2_hours)
    mv_1h_3h = moving_avg_diff(moving_average_1_hours, moving_average_3_hours)
    mv_3h_12h = moving_avg_diff(moving_average_3_hours, moving_average_12_hours)
    mv_12h_24h = moving_avg_diff(moving_average_12_hours, moving_average_24_hours)
    mv_24h_48h = moving_avg_diff(moving_average_24_hours, moving_average_48_hours)
    mv_48h_1w = moving_avg_diff(moving_average_48_hours, moving_average_1_week)
    mv_1w_1m = moving_avg_diff(moving_average_1_week, moving_average_1_month)
    
    print(len(mv_1w_1m))
    
    X = np.stack((mv_1h_2h, mv_1h_3h, mv_3h_12h, mv_12h_24h, mv_24h_48h, mv_48h_1w, mv_1w_1m), axis=1)
    X = X[min_period:len(X)-1]
    y = y[min_period:]
    
    print(len(X), len(y))
    
    return (X, y)

In [49]:
df = pd.read_csv("../data/clean/bitcoin_prices.csv")

X, y = get_X_y(df)

47698
0         NaN
1    0.000504
2    0.006687
3    0.003179
4   -0.001887
Name: Close, dtype: float64
47698
47699
47699
46953 46953
[[-1.05616696e-04  2.67662182e-04  7.08656390e-03 ... -6.23416896e-04
  -7.88400236e-02 -7.69663603e-02]
 [-1.69033130e-04 -2.95770482e-04  7.03496206e-03 ...  2.32957650e-03
  -7.93858293e-02 -7.73919751e-02]
 [-2.20264317e-03 -3.04723210e-03  5.15452953e-03 ...  2.58309768e-03
  -7.76370680e-02 -7.78228886e-02]
 ...
 [ 3.73658639e-03  5.40381903e-03  1.25110821e-02 ...  2.79762019e-02
   8.48901523e-02  1.79359449e-01]
 [-7.81360296e-03 -7.91614320e-03  1.08402128e-02 ...  2.65053926e-02
   8.58565323e-02  1.79456674e-01]
 [ 1.49980409e-03 -3.22717371e-03  8.60571264e-03 ...  2.47845570e-02
   8.69585770e-02  1.79584205e-01]]


In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)


clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)

SVC()

In [51]:
y_train_pred = clf.predict(X_train)
print("Train Accuracy:",metrics.accuracy_score(y_train, y_train_pred))

y_test_pred = clf.predict(X_test)
print("Test Accuracy:",metrics.accuracy_score(y_test, y_test_pred))

Train Accuracy: 0.5379871574798144
Test Accuracy: 0.5166828009035173
